In [1]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import torch
import pickle
from tqdm.notebook import tqdm

In [5]:
DATASET_DIR = "../quora-question-pairs-dataset/"

In [6]:
dataset_path = DATASET_DIR + "train.csv"

corpus_sentences = set()
with open(dataset_path) as fIn:
    reader = csv.DictReader(fIn)
    for row in tqdm(reader):
        for key in ['question1', 'question2']:
            que = row[key]
            if que:
                corpus_sentences.add(que)

corpus_sentences = list(corpus_sentences)

0it [00:00, ?it/s]

In [7]:
len(corpus_sentences)

537361

In [8]:
with open(DATASET_DIR + 'corpus_sentences.list', 'wb') as fp:
    pickle.dump(corpus_sentences, fp)

In [ ]:
with open (DATASET_DIR + 'corpus_sentences.list', 'rb') as fp:
    corpus_sentences = pickle.load(fp)

In [4]:
model_name = 'quora-distilbert-multilingual'
model = SentenceTransformer(model_name)

In [16]:
corpus_embeddings = model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True, num_workers=4, batch_size=1024)

Batches:   0%|          | 0/16793 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
torch.save(corpus_embeddings, DATASET_DIR+'corpus_embeddings.pt')

In [ ]:
corpus_embeddings = torch.load(DATASET_DIR+'corpus_embeddings.pt')

In [ ]:
def search(inp_question, top_k=10):
    start_time = time.time()
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    end_time = time.time()
    hits = hits[0]

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:5]:
        print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

In [ ]:
search("How can I learn Python online?")

In [ ]:
#German: How can I learn Python online?
search("Wie kann ich Python online lernen?")

In [ ]:
#Chinese: How can I learn Python online?
search("如何在线学习Python")